The aim of this analysis is to visualize all cities of israel and thier distance from their nearest hospital.

The wgs coordinates of all monicipal authorities in israel ('cities' for simplicity) can be obtain from the 'lamas_2018_wgs.csv' file that I have generated in previous notebooks (we will use it later on).

To find out the nearst hospital to each city, I first wanted to create a table of each hospital in Israel and its WGS coords.
On https://data.gov.il/dataset/satisfaction-hosp-general2016 I found a list of all hospitals in israel.
I took the hospitals list in hebrew and created an english name column. Asaf Harofe hospital was there twice, and Asuta Ashdod established later was missing - so i added it.

The WGS coords are obtained using GMAPS API.
Here I decided to lookup its hospital name in GMAPS API in hebrew


In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
file = r'..\csv\hospitals_empty.csv'
df = pd.read_csv(file, index_col = 0)

In [4]:
#The hospCode column is an arbitray , but I will keep it and use in later analyses.
df

,hebName,engName
hospCode,,
0,שיבא,Shiba
1,רמבם,Rambam
2,אסף הרופא,Asaf Harofe
3,וולפסון,Wolfson
4,זיו,Ziv
5,הלל יפה,Hilel Yafe
6,נהריה,Nahariya
7,ברזילי,Barzilay
8,פוריה,Poriya


In [5]:
#To run google API of any kind, go to "google cloud", enter a billing method and generate API for the task you need.
#note that this service costs money, but google give you free trials.
#If you not sure how jJust google "get GMAPS API credentiale"
#This is a simple function that gets a df of adressess and return same df with latitude and longitude WGS coordinates

from googlemaps import Client as GoogleMaps
import pandas as pd
import warnings
warnings.filterwarnings('ignore')



#define a func to work with
def getGWS(df):
    '''
    INPUT - a df with a 'name' column - to be used as adresss 
    OUTPUT - a df with 3 cols: name, lat and lon
    '''
    gmaps = GoogleMaps('Enter your key here')
    df= df.copy()
    df['lat'] = np.nan
    df['long'] = np.nan
    
    #Add the term 'hospital' in hebrew to help the GMAPS API
    df['nameIL'] = df['name'].apply(lambda x : ''.join(['בית החולים ', x]))    
    google_errors = []
    
       
    for x in range(0, len(df)):
        try:
            print(f'Starting round {x} of {len(df)-1}')
            geocode_result = gmaps.geocode(df['nameIL'][x])
            df['lat'][x] = geocode_result[0]['geometry']['location'] ['lat']
            df['long'][x] = geocode_result[0]['geometry']['location']['lng']
            print(df['lat'][x], df['long'][x])
        except:
            google_errors.append(df['name'][x])
    
    return df, google_errors


In [35]:
#hospitals = getGWS(df)[0]

Starting round 0 of 29
32.04671 34.842852
Starting round 1 of 29
32.8329785 34.9857062
Starting round 2 of 29
31.9660531 34.8394177
Starting round 3 of 29
32.0360309 34.7627074
Starting round 4 of 29
32.954157 35.492657
Starting round 5 of 29
32.451313 34.895778
Starting round 6 of 29
33.0083233 35.1171239
Starting round 7 of 29
31.6622785 34.5592871
Starting round 8 of 29
32.7523497 35.5386634
Starting round 9 of 29
32.0804971 34.7903713
Starting round 10 of 29
32.8058206 34.992719
Starting round 11 of 29
32.0886774 34.8667885
Starting round 12 of 29
31.2591051 34.8020335
Starting round 13 of 29
32.1823231 34.8954868
Starting round 14 of 29
31.8729453 34.8142255
Starting round 15 of 29
32.6210762 35.3165964
Starting round 16 of 29
32.7864002 34.9805234
Starting round 17 of 29
32.0886774 34.8667885
Starting round 18 of 29
29.5544708 34.9403539
Starting round 19 of 29
31.7649503 35.1494106
Starting round 20 of 29
31.79743629999999 35.2422844
Starting round 21 of 29
32.7021261 35.2923843

In [70]:
#drop the concatenate name
hospitals.drop(columns = ['nameIL'], inplace = True)

In [4]:
hospitals.head()

In [75]:
#hospitals.to_csv(r'..\csv\hospitals.csv')